In [15]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from matplotlib import pyplot as plt
from scipy import signal
from mpl_toolkits.mplot3d import Axes3D
import ipyvolume as ipv
import matplotlib.colors as colors
import matplotlib.image as img

#from skimage.filters import gaussian

%matplotlib inline

In [16]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [17]:
def k_():
    return 2*np.pi
def L_(a):
    return 20
def passo_(a):
    return 0.07


def grid(a):
    L = L_(a)
    passo = passo_(a)
    x = np.arange(-L,L,passo)
    y = np.arange(-L,L,passo)
    
    xx, yy = np.meshgrid(x, y)
    return xx, yy

In [18]:
def numero_fresnel(a, z):
    return a**2*k_()/(2*np.pi*z)

In [19]:
def h(a, z):
    k = k_()
    
    xx, yy = grid(a)

    return k*np.e**(1j*k*z)*np.e**(1j*k*(xx**2 + yy**2)/(2*z))/(1j*2*np.pi*z)

In [20]:
def E0(a, tipo):
    xx, yy = grid(a)
    if tipo == 1:
        return 1-(np.heaviside(np.sqrt(xx**2 + yy**2)-a, 0))
    if tipo == 2:
        return (np.heaviside(np.sqrt(xx**2 + yy**2)-a, 0))
    if tipo == 3:
        return ((np.heaviside(xx-a, 0))-(np.heaviside(xx+a, 0)))*((np.heaviside(yy-a, 0))-(np.heaviside(yy+a, 0)))
    if tipo == 4:
        return matriz_batman(a)

In [21]:
def matriz_batman(a):
    image = img.imread("bat.jpg")
    lum_img = image[:,:, 1]
    
    mat = np.zeros((1500, 1500))

    mat[850-721//2:850+722//2, :] = 1-lum_img/lum_img.max()
        
    step = int(len(mat)/(a/passo_(a)))
    mat = mat[::step, ::step]
    mat = mat.astype(int)
    
    dimb = len(mat)
    
    xx, yy = grid(a)
    dim = len(xx)
    mat2 = np.zeros((dim, dim))

    mat2[(dim - dimb)//2 : (dim + dimb)//2, (dim - dimb)//2 : (dim + dimb)//2] = mat

    return mat2

In [22]:
def difracao(z, a, tipo):
    L = L_(a)
    
    plt.style.use("classic")

    
    
    fig8 = plt.figure(constrained_layout=False)
    gs1 = fig8.add_gridspec(nrows=4, ncols=4, wspace=0.001)
    ax1 = fig8.add_subplot(gs1[0:-1, :])
    ax2 = fig8.add_subplot(gs1[-1, :-1])
    ax3 = fig8.add_subplot(gs1[-1, -1])


    cmap = plt.get_cmap('magma')
    new_cmap = truncate_colormap(cmap, 0., 1.)
    x, y = grid(a)
    padrao = signal.fftconvolve(E0(a, tipo), h(a, z), mode = "same")
    
    dim = len(padrao)
    padrao = padrao[dim//2-dim//3:dim//2+dim//3, dim//2-dim//3:dim//2+dim//3]
    x = x[dim//2-dim//3:dim//2+dim//3, dim//2-dim//3:dim//2+dim//3]
    y= y[dim//2-dim//3:dim//2+dim//3, dim//2-dim//3:dim//2+dim//3]
    E0_ = E0(a, tipo)
    E0_ = E0_[dim//2-dim//3:dim//2+dim//3, dim//2-dim//3:dim//2+dim//3]
        
    padrao_im = ax1.imshow(abs(padrao)/np.max(abs(padrao)), cmap= new_cmap)
    ax1.set_title("$F = $" + str(np.around(numero_fresnel(a,z), decimals=3)))
    fig8.colorbar(padrao_im, ax=ax1)
    ax1.axis('off')
    
    ax3.imshow(abs(E0_/np.max(abs(E0_))), cmap = "bone")
    ax3.axis('off')

        
    dim_cut = len(padrao)
            
    ax2.plot(x[dim_cut//2], abs(padrao)[:,dim_cut//2]/np.max(abs(padrao)[:,dim_cut//2]), color = "purple", linewidth = 2)
    ax2.plot(x[dim_cut//2], 0.8*np.heaviside(x[dim_cut//2]+a, 0) - 0.8*np.heaviside(x[dim_cut//2]-a, 0), color = "gray")
    ax2.set_xlabel("$x/\lambda$")
    ax2.set_ylabel("$I/I_{0}$")
   # ax2.savefig("intensi.pdf")
   # ax2.show()
    #plt.tight_layout()
    

 

In [25]:
tipo_ = widgets.Dropdown(
    options=[('Fenda Circular', 1), ('Ponto de Arago', 2), ('Fenda Quadrada', 3), ('Batman', 4)],
    value=1,
    description='Fenda:')

interact_manual(difracao,z= 20, a=widgets.FloatSlider(value = 3
, step=0.1, max = 20), tipo = tipo_);



interactive(children=(IntSlider(value=20, description='z', max=60, min=-20), FloatSlider(value=3.0, descriptio…